In [9]:
import os
from fastai.vision.all import *
import zipfile

import params as params


import wandb

from kaggle.api.kaggle_api_extended import KaggleApi


In [10]:
PATH_TO_DATA = "data"
DATASET_NAME = 'brian-tumor-dataset'
UNZIPPED_NAME = 'Brain Tumor Data Set'
TUMOR_SUB_DIR = 'Brain Tumor'
HEALTHY_SUB_DIR = 'Healthy'

In [11]:
#Get files from kaggle
kaggle_api = KaggleApi()
kaggle_api.authenticate()

if os.path.exists(PATH_TO_DATA) == False:
    os.mkdir(PATH_TO_DATA)
    
kaggle_api.dataset_download_files(os.path.join('preetviradiya',"brian-tumor-dataset"), path=PATH_TO_DATA)

In [12]:
# Unzip
with zipfile.ZipFile(os.path.join(PATH_TO_DATA, DATASET_NAME + ".zip"), 'r') as zip_ref:
    zip_ref.extractall(os.path.join(PATH_TO_DATA))


In [13]:

def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

def get_classes_per_image(mask_data, class_labels):
    unique = list(np.unique(mask_data))
    result_dict = {}
    for _class in class_labels.keys():
        result_dict[class_labels[_class]] = int(_class in unique)
    return result_dict

def _create_table(image_files, classification_label):
    "Create a table with the dataset"
    table = wandb.Table()
    
    for i, image_file in progress_bar(enumerate(image_files), total=len(image_files)):
        image = Image.open(image_file)
        table.add_data(
            str(image_file.name),
            wandb.Image(
                    image,
            ),
            str(classification_label)
        )
    

In [14]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="upload")
raw_data_at = wandb.Artifact(params.RAW_DATA_AT, type="raw_data")

In [15]:
tumour_files = get_image_files(os.path.join(PATH_TO_DATA,UNZIPPED_NAME,UNZIPPED_NAME,TUMOR_SUB_DIR), recurse=False)
healthy_files = get_image_files(os.path.join(PATH_TO_DATA,UNZIPPED_NAME,UNZIPPED_NAME,HEALTHY_SUB_DIR), recurse=False)

In [24]:
# Create labels
labels = ["0"] * len(healthy_files)
labels.append(["1"] * len(tumour_files))

# Generate files list
all_files = tumour_files.copy()
all_files.append(healthy_files)

# Create table
image_table = _create_table(all_files, labels)

raw_data_at.add(image_table, "eda_table")
run.log_artifact(raw_data_at)
run.finish()

AttributeError: 'list' object has no attribute 'read'